# **Judgements of SCP - RAG - Main Notebook**

### Submitted by: Abdul Haseeb (17132), Annayah Usman (18673), Dr. Sawera Hanif (29413)

## **Downloading Dependencies**

In [ ]:
pip install --upgrade pip # For Annayah

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install -q langchain_community langchain_pinecone accelerate tiktoken transformers_stream_generator einops optimum auto-gptq bitsandbytes sentence-transformers

In [ ]:
!pip install -U bitsandbytes rank_bm25 ipython-autotime

In [ ]:
# !pip install -q langchain_community langchain_pinecone accelerate tiktoken transformers_stream_generator einops optimum auto-gptq bitsandbytes sentence-transformers --no-deps

In [ ]:
# !pip install -q langchain

time: 279 µs (started: 2024-11-14 10:14:42 +00:00)


## **Importing Dependencies**

In [ ]:
%load_ext autotime

import csv

import sys

import numpy as np

import pandas as pd

from langchain.schema import Document

from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import AutoModel, AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

from transformers.generation import GenerationConfig

from langchain.embeddings import HuggingFaceEmbeddings

import os

import getpass

import time

from pinecone import Pinecone

from langchain_pinecone import PineconeVectorStore

import torch

import optimum

from huggingface_hub import login

from langchain.retrievers import BM25Retriever, EnsembleRetriever

from langchain.prompts import PromptTemplate

from langchain_community.document_transformers import (
    LongContextReorder,
)

import warnings

warnings.filterwarnings("ignore")

import textwrap

time: 19.7 s (started: 2024-11-16 20:16:21 +00:00)


## **1. Loading Corpus**

In [ ]:
df=pd.read_csv("/kaggle/input/scp-judgements-cleaned-with-metadata-csv/SCP_Judgements_Cleaned_with_Metadata.csv")

df.head()

,case subject,case no,case title,author judge,judgment date,content
0,cases relating to pemra /.,c.a.11/2022,pakistan electronic media regulatory authority...,mr. justice munib akhtar,10-11-2022,(appellate jurisdiction)\n\n\npresent:\nmr. ju...
1,suit for declaration/possession,c.a.17-q/2023,noorullah & others v. ghulam murtaza & others,mr. justice syed hasan azhar rizvi,10-11-2023,c;\nit\ncases\n\n(appellate jurisdiction)\npre...
2,service/against reinstatment into service,c.a.19/2022,government of khyber pakhtunkhwa through chief...,mr. justice muhammad ali mazhar,06-10-2022,(appellate jurisdiction)\n\n\npresent:\nmr. ju...
3,suit for declaration/cancellation of document,c.a.2015/2022,syed zahid hussain shah v. mumtaz ali and others,mr. justice qazi faez isa,09-05-2023,(appellate jurisdiction)\n\n\npresent:\nmr. ju...
4,customs/confiscation of vehicle,c.a.2016/2022,"m/s middle east construction company, karachi ...",mr. justice qazi faez isa,16-02-2023,(appellate jurisdiction)\n\npresent:\n\nmr. ju...


time: 138 ms (started: 2024-11-16 20:16:50 +00:00)


## **2. Splitting Text**

In [ ]:
# Content Length

content_length=[]

for i in df['content']:

  content_length.append(len(i))



# Metadata Length

metadata_length=[]

for i in range(len(df)):

  metadata_length.append(len(df['case subject'][i]+df['case no'][i]+df['case title'][i]+df['author judge'][i]+df['judgment date'][i]))



# Filter content length list less than 100000

content_length=[i for i in content_length if i<100000]



# Proportion of lengths in content_length list which is below average character length

b_avg=len([i for i in content_length if i<np.mean(content_length)])/len(content_length)*100



# Proportion of lengths in content_length list which is above 2500 and below 5000 character length

b_5k=len([i for i in content_length if i<5000])/len(content_length)*100



output=f'''

Chunking Strategy:

- Average Number of Content Characters (Excluding 4 Outliers): {round(np.mean(content_length))}

- Majority of The Documents are Below Average Character Length: {round(b_avg)}%

- Splitting Average Character Length in 5 Chunks Gives a Split of: {round(np.mean(content_length)/5)}

- Therefore, chunk size of 2500 characters with 10% overlap seems a good balance for retrieval.

- Smaller documents would be retrieved completely in 1-2 chunks and rest would be irrelevant that we can tell the LLM to ignore.

- Very few small documents would be covered in 1-2 chunks as proportion of documents with below 5000 characters is: {round(b_5k)}%

- Key areas of large documents would be covered in 3-5 chunks, otherwise we can always increase retrieval space for very large documents.

- A bigger chunk size than a very small one like 1000 characters also has the advantage of generic querying!

- We often wanna summarize what happened in a particular case rather than knowing just one statement from the judgement. But even some aspects of the judgement would be covered in this chunk size as well.

- A 1,000-character chunk could make retrieval too fragmented, as legal orders and judgments are often longer passages that may not fit well within 1,000 characters.



Conclusion:

- So 2500 characters offers a good balance between Granularity and Contextual Coherence!



Embedding Model Considerations:

- While our embedding model dimension of 1024 would work well for capturing the nuanced information in legal text.

- The max token limit of 512 tokens could be a limitation. Going with the rule of thumb of 1 token ~ 4 chars then a chunk should have 2048 characters.

- However, our strategy here is to first split the documents and then add metadata of document to each of its chunks.

- The average metadata string size (rounded up to nearest hundred) is: {round(round(np.mean(metadata_length),-2))}



Thus, for splitting we'll keep chunk size to 1800 with 200 characters overlap such that after including metadata chunk size is 2000

Keeping this in view the retrieval should be with top_k=[5,10] as (Average Number of Content Characters)/1800 = {round(np.mean(content_length)/1800)}



Type of Splitter:

- Our documents lack subsections and follow a consistent structure: Bench Member Names > Petition No > Petitioner/Appellate versus. Respondent Names > Representative Names > Judgement/Order.

- The Judgment/Order section is fairly uniform and continuous.

- Therefore a character/token based splitter would be a simpler and effective approach than a semantic based splitter.

- As our documents have a consistent structure and lack subsections, semantic chunking offers limited additional value.

- Besides, character based splitting with overlapping would allow context continuity along with meeting embedding model’s token limit.



Note: Split is done row-wise, preserving individual judgments as separate documents.

'''



print(output)



Chunking Strategy:

- Average Number of Content Characters (Excluding 4 Outliers): 13874

- Majority of The Documents are Below Average Character Length: 67%

- Splitting Average Character Length in 5 Chunks Gives a Split of: 2775

- Therefore, chunk size of 2500 characters with 10% overlap seems a good balance for retrieval.

- Smaller documents would be retrieved completely in 1-2 chunks and rest would be irrelevant that we can tell the LLM to ignore.

- Very few small documents would be covered in 1-2 chunks as proportion of documents with below 5000 characters is: 14%

- Key areas of large documents would be covered in 3-5 chunks, otherwise we can always increase retrieval space for very large documents.

- A bigger chunk size than a very small one like 1000 characters also has the advantage of generic querying!

- We often wanna summarize what happened in a particular case rather than knowing just one statement from the judgement. But even some aspects of the judgement would be 

In [ ]:
# Initialize the text splitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1800, chunk_overlap=200)



# Create a list to hold all document chunks

documents = []



# Process each row to split content and add metadata inside page_content

for _, row in df.iterrows():

    content = row['content']

    metadata = {

        "case_subject": row['case subject'],

        "case_no": row['case no'],

        "case_title": row['case title'],

        "author_judge": row['author judge'],

        "judgment_date": row['judgment date']

    }



    # Prepare a metadata string to insert into page_content

    metadata_str = (

        f"Case Subject: {metadata['case_subject']}\n"

        f"Case No: {metadata['case_no']}\n"

        f"Case Title: {metadata['case_title']}\n"

        f"Author Judge: {metadata['author_judge']}\n"

        f"Judgment Date: {metadata['judgment_date']}\n\n"

    )



    # Split content into chunks

    chunks = text_splitter.split_text(content)



    # Create a Document instance for each chunk with metadata embedded in the page_content

    for chunk in chunks:

        # Combine metadata string and chunk content

        chunk_with_metadata = metadata_str + chunk

        documents.append(Document(page_content=chunk_with_metadata))

time: 165 ms (started: 2024-11-16 20:16:53 +00:00)


In [ ]:
# Manually checked that chunks with less than 370 characters have no meaningful content and character length of this chunk is inflated with the inclusion of metadata in the chunk string!

# Therefore filtering out chunks with character size less than 370 characters.

documents=[i for i in documents if len(i.page_content)>370]

time: 2.03 ms (started: 2024-11-16 20:16:54 +00:00)


In [ ]:
print(f'Total Chunks: {len(documents)}')

Total Chunks: 3541
time: 478 µs (started: 2024-11-16 20:16:56 +00:00)


**Metadata Handling**

* Each judgement content is split into smaller, manageable chunks, with a specified chunk size of 1800 characters and an overlap of 200 characters between consecutive chunks through RecursiveCharacterTextSplitter.

* For each judgement its relevant metadata such as the case subject, case number, case title, author judge, and judgment date, is formatted into a metadata string.

* For each chunk its respective metadata string is embedded in it.



<br/>



* The above strategy to handle metadata is devised rather than keyword or fuzzy filtering because:

  * The key advantage is that when referring to a specific case in the query, it is assumed that the user will naturally include a reference to the case. This approach avoids the limitations of filter-based queries, where metadata filters are rigid and may miss subtle case variations.

    * Filter-based Query: ("What was supreme court's judgement regarding section 63 of constitution?", metadata filters: {title:"A versus B"})

    * Metadata Embedded Query: "In A versus B what was supreme court's judgement regarding section 63 of constitution?"

  * While fuzzy filtering can handle spelling mistakes and approximate matches, semantic search offers a deeper understanding of the context and meaning behind the query. By embedding metadata in the chunks, semantic search can retrieve more accurate and relevant results, even if the query contains errors, as it focuses on the overall meaning rather than just the specific spelling or keywords.

  * In filter-based methods, the user must be aware of the metadata tags available in the database and construct queries accordingly. In contrast, with semantic search, there's no need for the user to know or specify metadata tags. The system can automatically interpret and match the query with relevant information, making the search process more intuitive and user-friendly.

## **3. Loading Embedding Model**

In [ ]:
model_name = "dunzhang/stella_en_1.5B_v5"

embeddings = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/174k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

2_Dense_1024/config.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/6.30M [00:00<?, ?B/s]

time: 2min 37s (started: 2024-11-16 20:16:58 +00:00)


The above embedding model is selected because:



* 1024-dimensional embeddings: This high-dimensional embedding captures nuanced information well, especially useful for legal text.



* Proven Performance: Ranked third on the METB leaderboard, ensuring strong performance on English documents.



* Efficient for RAG Setup: Provides high-quality embeddings with minimal latency due to its smaller size compared to other top models. Only 1.5 billion parameters.

## **4. Initializing VectorDb and Ingesting**

In [ ]:
# pinecone.api: pinecone.api



if not os.getenv("PINECONE_API_KEY"):

    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")



pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

index_name = "scp-vdb"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]



if index_name not in existing_indexes:

    pc.create_index(

        name=index_name,

        dimension=1024,

        metric="cosine",

        spec=ServerlessSpec(cloud="aws", region="us-east-1"),

    )

    while not pc.describe_index(index_name).status["ready"]:

        time.sleep(1)

index = pc.Index(index_name)

Enter your Pinecone API key:  ········


time: 8.87 s (started: 2024-11-16 20:19:48 +00:00)


In [ ]:
vector_store = PineconeVectorStore(index=index, embedding=embeddings)

time: 403 µs (started: 2024-11-16 20:20:00 +00:00)


In [ ]:
# NOTE: Use GPU to do this otherwise it'll take forever! (Currently taking 22 mins)

# NOTE: RUN THIS CELL ONCE ONLY OTHERWISE IT'LL CREATE DUPLICATE VECTOR EMBEDDINGS!

# Once stored, we only have to load embedding model to embed query and connect to VectorDb from above!



####vector_store.add_documents(documents=documents)

## **RAG - Experimentation (Keeping LLM Fixed - Llama 3.1 8B Instruct)**

In [ ]:
login(token='HF_api')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful
time: 43.9 ms (started: 2024-11-16 20:20:03 +00:00)


In [ ]:
# Note: The default behavior now has injection attack prevention off.

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(

    "meta-llama/Meta-Llama-3.1-8B-Instruct",

    device_map="auto",

    trust_remote_code=True,

    torch_dtype=torch.bfloat16,

    quantization_config=BitsAndBytesConfig(load_in_8bit=True)

).eval()

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

time: 7min 43s (started: 2024-11-16 20:20:06 +00:00)


### **4. Retrievers**

### **4.1. Similarity Retriever**

#### **4.1.1. Generic Query using Similarity Retriever**

In [ ]:
generic_query='''

What was court's judgement on the following income tax case:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

generic_query=generic_query.lower()



context = vector_store.similarity_search_with_score(generic_query,k=10)



prompt=f'''

context:

{context[0][0].page_content}\n\n

{context[1][0].page_content}\n\n

{context[2][0].page_content}\n\n

{context[3][0].page_content}\n\n

{context[4][0].page_content}\n\n

{context[5][0].page_content}\n\n

{context[6][0].page_content}\n\n

{context[7][0].page_content}\n\n

{context[8][0].page_content}\n\n

{context[9][0].page_content}



Suppose that you're a legal expert and then based on the above context answer the following query:



{generic_query}



Note: Only use the context that is given above to answer the query. Not all of the context might be important so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context! and mention your

reasons for saying this.

'''



messages = [{"role": "user", "content": f'{prompt}'}]

pipe = pipeline("text-generation",model=model,tokenizer=tokenizer)

generation_args = {"max_new_tokens": 2000,

                   "return_full_text": False,

                   "temperature": 0.0,

                   "do_sample": False}

output = pipe(messages, **generation_args)

print(textwrap.fill(output[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the court's judgment in the case of M/s Islamabad
Electric Supply Company Limited (IESCO) through its Finance Director Islamabad
v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its
Chairman Islamabad and others (Case No: C.P.L.A.1920/2022) was that the civil
petitions were dismissed and leave was refused.  The court did not find any
illegality or perversity in the impugned judgment of the learned high court. The
judgment was delivered by Mr. Justice Muhammad Ali Mazhar on 10-08-2022.  The
court's decision was based on the fact that the tax authority had afforded ample
opportunities of hearing to the petitioner and had complied with the
requirements of due process of law before passing the orders under section 161
of the ordinance.
time: 53.3 s (started: 2024-11-14 10:48:02 +00:00)


#### **4.1.2. Specific Query using Similarity Retriever**

In [ ]:
specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()



context = vector_store.similarity_search_with_score(specific_query,k=10)



prompt=f'''

context:

{context[0][0].page_content}\n\n

{context[1][0].page_content}\n\n

{context[2][0].page_content}\n\n

{context[3][0].page_content}\n\n

{context[4][0].page_content}\n\n

{context[5][0].page_content}\n\n

{context[6][0].page_content}\n\n

{context[7][0].page_content}\n\n

{context[8][0].page_content}\n\n

{context[9][0].page_content}



Suppose that you're a legal expert and then based on the above context answer the following query:



{specific_query}



Note: Only use the context that is given above to answer the query. Not all of the context might be important so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context! and mention your

reasons for saying this."

'''



messages = [{"role": "user", "content": f'{prompt}'}]

pipe = pipeline("text-generation",model=model,tokenizer=tokenizer)

generation_args = {"max_new_tokens": 2000,

                   "return_full_text": False,

                   "temperature": 0.0,

                   "do_sample": False}

output = pipe(messages, **generation_args)

print(textwrap.fill(output[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided context, section 177 of the income tax ordinance is
related to the audit exercise. According to the context, section 177 of the
ordinance provides that the commissioner may call for any record or documents,
including books of account maintained under this ordinance, for conducting audit
of the income tax affairs of a person. However, the commissioner must record
reasons in writing and communicate these reasons to the taxpayer while calling
for the record or documents.  The context also mentions that section 177 of the
ordinance is a precondition for taking any adverse action under sections 161 and
205. However, it is stated that the provisions of section 177 and section 161
are independent and self-governing, and the commissioner may amend or further
amend an order of recovery under section 161 without initiating proceedings
under section 177.  In the context of the case, m/s Islamabad Electric Supply
Company Limited (IESCO) through its Finance Director Islamabad 

#### **4.1.3. Completely Generic Query using Similarity Retriever**

**Limitation: RAG with similarity search doesn't do well without metadata inclusion (Poor Retrieval)**

In [ ]:
completely_generic_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205?

Which cases can you reference for this?

'''

completely_generic_query=completely_generic_query.lower()



context = vector_store.similarity_search_with_score(completely_generic_query,k=10)



prompt=f'''

context:

{context[0][0].page_content}\n\n

{context[1][0].page_content}\n\n

{context[2][0].page_content}\n\n

{context[3][0].page_content}\n\n

{context[4][0].page_content}\n\n

{context[5][0].page_content}\n\n

{context[6][0].page_content}\n\n

{context[7][0].page_content}\n\n

{context[8][0].page_content}\n\n

{context[9][0].page_content}



Suppose that you're a legal expert and then based on the above context answer the following query:



{completely_generic_query}



Note: Only use the context that is given above to answer the query. Not all of the context might be important so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context! and mention your

reasons for saying this."

'''



messages = [{"role": "user", "content": f'{prompt}'}]

pipe = pipeline("text-generation",model=model,tokenizer=tokenizer)

generation_args = {"max_new_tokens": 2000,

                   "return_full_text": False,

                   "temperature": 0.0,

                   "do_sample": False}

output = pipe(messages, **generation_args)

print(textwrap.fill(output[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I don't know as I don't have sufficient context!  The query is about the
implications of section 177 of the income tax ordinance regarding the audit
exercise and its relation to sections 161 and 205. However, the provided context
does not mention section 177, 161, or 205. The context only discusses sections
111, 122, and other related provisions of the income tax ordinance. Therefore, I
do not have enough information to provide a relevant answer.
time: 35.2 s (started: 2024-11-14 10:50:53 +00:00)


**Extra Experiment: Another Attempt (Again Poor Retrieval)**

In [ ]:
completely_generic_query='''

Give case title of case discuss section 124, section 160, section 161, section 165, section 177 & section 205 of the Income Tax Ordinance 2001!

'''

completely_generic_query=completely_generic_query.lower()



context = vector_store.similarity_search_with_score(completely_generic_query,k=10)



prompt=f'''

context:

{context[0][0].page_content}\n\n

{context[1][0].page_content}\n\n

{context[2][0].page_content}\n\n

{context[3][0].page_content}\n\n

{context[4][0].page_content}\n\n

{context[5][0].page_content}\n\n

{context[6][0].page_content}\n\n

{context[7][0].page_content}\n\n

{context[8][0].page_content}\n\n

{context[9][0].page_content}



Suppose that you're a legal expert and then based on the above context answer the following query:



{completely_generic_query}



Note: Only use the context that is given above to answer the query. Not all of the context might be important so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your

reasons for saying this.

'''



messages = [{"role": "user", "content": f'{prompt}'}]

pipe = pipeline("text-generation",model=model,tokenizer=tokenizer)

generation_args = {"max_new_tokens": 2000,

                   "return_full_text": False,

                   "temperature": 0.0,

                   "do_sample": False}

output = pipe(messages, **generation_args)

print(textwrap.fill(output[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


I don't know as I don't have sufficient context!  The query asks for the case
title of a case that discusses sections 124, 160, 161, 165, 177, and 205 of the
Income Tax Ordinance 2001. However, the provided context only discusses sections
111, 122, and 122(9) of the Income Tax Ordinance 2001, and does not mention any
of the sections mentioned in the query. Therefore, I do not have enough
information to answer the query.
time: 37.3 s (started: 2024-11-14 10:51:28 +00:00)


### **4.2. BM25 Retriever**

#### **4.2.1. Generic Query using BM25 Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

generic_query = '''
What was court's judgement on the following income tax case:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022
'''

generic_query = generic_query.lower()

bm25_context = bm25_retriever.invoke(generic_query, k=10)

bm25_prompt = f"""
context:
{''.join([doc.page_content for doc in bm25_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{generic_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_bm25 = [{"role": "user", "content": bm25_prompt}]
output_bm25 = pipe(messages_bm25, **generation_args)

print(textwrap.fill(output_bm25[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the court's judgment in the income tax case "m/s
islamabad electric supply company limited (iesco) through its finance director
islamabad v. the appellate tribunal inland revenue (headquarter), islamabad
through its chairman islamabad and others" (Case No: c.p.l.a.1920/2022) was that
the learned high court and the learned appellate tribunal inland revenue failed
to consider that the order under sections 161 and 205 of the ordinance cannot be
passed without initiating proceedings under section 177 of the ordinance.
However, the context does not explicitly state the final judgment of the court
in the case. It only mentions that the learned high court answered the question
of law framed in the tax references in the negative, while upholding the order
passed by the learned appellate tribunal inland revenue.  Therefore, the final
judgment of the court in the case is not explicitly stated in the given context.
time: 49.2 s (started: 2024-11-14 10:35:54 +00:00)


#### **4.2.2. Specific Query using BM25 Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

bm25_context = bm25_retriever.invoke(specific_query, k=10)

bm25_prompt = f"""
context:
{''.join([doc.page_content for doc in bm25_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{specific_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_bm25 = [{"role": "user", "content": bm25_prompt}]
output_bm25 = pipe(messages_bm25, **generation_args)

print(textwrap.fill(output_bm25[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the implications of section 177 of the income tax
ordinance regarding the audit exercise and its relation to sections 161 and 205
are as follows:  Section 177 of the ordinance is predominantly related to the
audit exercise, where the commissioner may call for any record or documents,
including books of account, for conducting an audit of the income tax affairs of
a person. However, this can only be done after recording reasons in writing and
communicating those reasons to the taxpayer. Moreover, no record can be called
after the expiry of six years from the end of the tax year to which they relate.
The learned counsel for the petitioner argued that the orders under sections 161
and 205 of the ordinance cannot be passed without initiating proceedings under
section 177 of the ordinance, which is a precondition before taking any adverse
action under sections 161 and 205. This implies that section 177 is a
prerequisite for the exercise of powers under sections 1

#### **4.2.3. Completely Generic Query using BM25 Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

completely_generic_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205?

Which cases can you reference for this?

'''

completely_generic_query=completely_generic_query.lower()

bm25_context = bm25_retriever.invoke(completely_generic_query, k=10)

bm25_prompt = f"""
context:
{''.join([doc.page_content for doc in bm25_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{completely_generic_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_bm25 = [{"role": "user", "content": bm25_prompt}]
output_bm25 = pipe(messages_bm25, **generation_args)

print(textwrap.fill(output_bm25[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the implications of section 177 of the income tax
ordinance regarding the audit exercise and its relation to sections 161 and 205
are as follows:  Section 177 of the ordinance is predominantly related to the
audit exercise, where the commissioner may call for any record or documents,
including books of account maintained under this ordinance for conducting audit
of the income tax affairs of a person. However, the legislature has not put into
effect any precondition under section 177 of the ordinance to embark on an audit
exercise first, and then start off proceedings under section 161 of the
ordinance.  The learned judge, Mr. Justice Muhammad Ali Mazhar, held that the
intention of the legislature must be discovered from the words used. If the
words used are capable of one construction only, then it would not be open to
the courts to adopt any other hypothetical construction. In this case, the words
of section 177 of the ordinance are readily understood witho

### **4.3. Ensemble Retriever**

#### **4.3.1. Generic Query using Ensemble Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

generic_query = '''
What was court's judgement on the following income tax case:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022
'''

generic_query = generic_query.lower()

ensemble_context = ensemble_retriever.invoke(generic_query, k=10)

ensemble_prompt = f"""
context:
{''.join([doc.page_content for doc in ensemble_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{generic_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the court's judgment in the income tax case of M/s
Islamabad Electric Supply Company Limited (IESCO) through its Finance Director
Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad
through its Chairman Islamabad and others (Case No: C.P.L.A.1920/2022) was that
the Appellate Tribunal Inland Revenue was justified in passing the order under
Section 161/205 of the Income Tax Ordinance, 2001, without proceedings under
Section 177 of the Ordinance.  The court, in its judgment, held that the
Appellate Tribunal Inland Revenue had provided sufficient opportunities to the
taxpayer (IESCO) to provide relevant documents or details, but they failed to do
so. Therefore, the order was passed on the basis of the available record.  The
court also framed a common question of law in the reference applications, which
was:  "Whether under the facts and circumstances of the case, was the respondent
No.1 justified to hold proceedings and pass order under S

#### **4.3.2. Specific Query using Ensemble Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

ensemble_prompt = f"""
context:
{''.join([doc.page_content for doc in ensemble_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{specific_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


According to the context, section 177 of the income tax ordinance is related to
the audit exercise, where the commissioner may call for any record or documents,
including books of account maintained under this ordinance, for conducting audit
of the income tax affairs of a person. However, this section is not a
precondition for the exercise of powers under sections 161 and 205.  The context
states that section 177 of the ordinance is "predominantly related to the audit
exercise, whereby the commissioner may call for any record or documents
including books of account maintained under this ordinance for conducting audit
of the income tax affairs of a person but after recording reasons in writing
and, as a precondition, the said reasons shall be communicated to the taxpayer
while calling for the record or documents including books of accounts of the
taxpayer."  Furthermore, the context clarifies that the provisions under section
177 of the ordinance do not override or overlap the provision

#### **4.3.3. Completely Generic Query using Ensemble Retriever**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

completely_generic_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205?

Which cases can you reference for this?

'''

completely_generic_query=completely_generic_query.lower()

ensemble_context = ensemble_retriever.invoke(completely_generic_query, k=10)

ensemble_prompt = f"""
context:
{''.join([doc.page_content for doc in ensemble_context])}

Suppose that you're a legal expert and then based on the above context answer the following query:

{completely_generic_query}

Note: Only use the context that is given above to answer the query. Not all of the context might be important, so only use what you find related to the query.

If the context has nothing that is directly related to the query or not sufficient, say that "I don't know as I don't have sufficient context!" and mention your reasons for saying this.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the given context, the implications of section 177 of the income tax
ordinance regarding the audit exercise and its relation to sections 161 and 205
are as follows:  Section 177 of the ordinance is predominantly related to the
audit exercise, where the commissioner may call for any record or documents,
including books of account maintained under this ordinance for conducting audit
of the income tax affairs of a person. However, the proviso to sub-section (1)
clarified that no record shall be called after expiry of six years from the end
of the tax year to which they relate.  The learned counsel for the petitioner in
the case of M/s Islamabad Electric Supply Company Limited (IESCO) through its
Finance Director Islamabad v. The Appellate Tribunal Inland Revenue
(Headquarter), Islamabad through its Chairman Islamabad and others (Case No:
c.p.l.a.1920/2022) argued that the order under sections 161 and 205 of the
ordinance cannot be passed without initiating proceedings under secti

### **Observations:**

We went with Ensemble Retriever that combines both keyword based retriever (BM25) and semantic search (similarity search) by assigning weights to their results. It aggregates the scores from both retrievers to rank and return the most relevant documents, leveraging the strengths of both sparse and dense retrieval methods for better accuracy.

### **5. Generation**

* For the generation enhancement experimentation ahead, we used the Specific Query to check different strategies and then applied final best strategy for comparison between LLMs to all the queries: Generic, Specific, Completely Generic.

* We are using Ensemble Retrieval throughout this section and beyond.

* We attempted first having the summary generated from the context and query, and then with summary generated from context only i.e. without the query. We selected the latter approach because it would generate summaries based on the document content instead of having bias from the query.

* When using each of the summaries along with the context to get the query answer from RAG, we decide the approach of summary without query. This isolates the extracting relevant information phase from the generation phase, resulting in a summary that is more of an abstract of the document and then would answer query based of of that without introducing too many details which would help this strategy to generalize to all query types.

* The final approach is to use Long Context Reordering and integrate it with the above approaches. We first retrive the Ensemble Context, reorder it, then pass the Reordered Context to generate the summary without the query. Then use that summary along with the reordered context to answer the query. And this is the final strategy used across all 3 queries in the 2 LLMs for comparison of LLMs and Final RAG Evaluation.

#### **5.1.1. Generating Summary (With Query)**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

ensemble_prompt = f"""
context:
{''.join([doc.page_content for doc in ensemble_context])}

Based on the above context, summarize the key implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205. Use the case provided as a reference.

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

{specific_query}

Note: Only use the context above to answer the query. If the context is insufficient, say "I don't know as I don't have sufficient context!" and explain why.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

summary1_w_query = output_ensemble[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided context, the key implications of Section 177 of the Income
Tax Ordinance regarding the audit exercise and its relation to Sections 161 and
205 are as follows:  1. **Independence of Sections 177, 161, and 205**: Section
177 of the ordinance is related to the audit exercise, where the commissioner
may call for any record or documents for conducting audit of the income tax
affairs of a person. However, the audit exercise under Section 177 is
independent of the provisions contained under Section 161 of the ordinance. This
means that the commissioner may initiate proceedings under Section 161 without
first conducting an audit under Section 177.  2. **No Precondition for Section
161**: The audit exercise under Section 177 is not a precondition for the
commissioner to initiate proceedings under Section 161. The commissioner may
amend or further amend an order of recovery under Section 161 without first
conducting an audit under Section 177.  3. **Different Premise for Se

#### **5.1.2. Using Summary (With Query) to Answer Query**

In [ ]:
ensemble_prompt = f"""
Initial Summary:
{summary1_w_query}

context:
{''.join([doc.page_content for doc in ensemble_context])}

Suppose that you're a legal expert and based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the context and summary provided. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided context and initial summary, the key implications of
Section 177 of the Income Tax Ordinance regarding the audit exercise and its
relation to Sections 161 and 205 are as follows:  1. **Independence of Sections
177, 161, and 205**: Section 177 of the ordinance is related to the audit
exercise, where the commissioner may call for any record or documents for
conducting audit of the income tax affairs of a person. However, the audit
exercise under Section 177 is independent of the provisions contained under
Section 161 of the ordinance.  2. **No Precondition for Section 161**: The audit
exercise under Section 177 is not a precondition for the commissioner to
initiate proceedings under Section 161. The commissioner may amend or further
amend an order of recovery under Section 161 without first conducting an audit
under Section 177.  3. **Different Premise for Section 161**: Section 161 of the
ordinance has a different premise than Section 177. The commissioner may amen

#### **5.2.1. Generating Summary (Without Query)**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

ensemble_prompt = f"""
context:
{''.join([doc.page_content for doc in ensemble_context])}

Based on the above context, summarize the key implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205. Use the case provided as a reference.

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

Note: Only use the context above to answer the query. If the context is insufficient, say "I don't know as I don't have sufficient context!" and explain why.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

summary2_wo_query = output_ensemble[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided context, the key implications of Section 177 of the Income
Tax Ordinance regarding the audit exercise and its relation to Sections 161 and
205 are as follows:  1. **Section 177 is related to the audit exercise**:
Section 177 of the ordinance is predominantly related to the audit exercise,
where the commissioner may call for any record or documents, including books of
account, for conducting an audit of the income tax affairs of a person.  2.
**Precondition for audit**: The audit under Section 177 requires the
commissioner to record reasons in writing and communicate them to the taxpayer
while calling for the record or documents.  3. **Independence of Sections 177,
161, and 205**: The provisions of Section 177 do not override or overlap the
provisions of Sections 161 and 205. Both provisions are independent and have
self-governing corollaries.  4. **No requirement for Section 177 compliance for
Section 161 and 205**: The course of action and benchmark enumerated un

#### **5.2.2. Using Summary (Without Query) to Answer Query**

In [ ]:
ensemble_prompt = f"""
Initial Summary:
{summary2_wo_query}

context:
{''.join([doc.page_content for doc in ensemble_context])}

Suppose that you're a legal expert and based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the context and summary provided. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

generation_args = {
    "max_new_tokens": 2000,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_ensemble = [{"role": "user", "content": ensemble_prompt}]
output_ensemble = pipe(messages_ensemble, **generation_args)

print(textwrap.fill(output_ensemble[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Based on the provided context and summary, the key implications of Section 177
of the Income Tax Ordinance regarding the audit exercise and its relation to
Sections 161 and 205 are as follows:  1. **Section 177 is related to the audit
exercise**: Section 177 of the ordinance is predominantly related to the audit
exercise, where the commissioner may call for any record or documents, including
books of account, for conducting an audit of the income tax affairs of a person.
2. **Precondition for audit**: The audit under Section 177 requires the
commissioner to record reasons in writing and communicate them to the taxpayer
while calling for the record or documents.  3. **Independence of Sections 177,
161, and 205**: The provisions of Section 177 do not override or overlap the
provisions of Sections 161 and 205. Both provisions are independent and have
self-governing corollaries.  4. **No requirement for Section 177 compliance for
Section 161 and 205**: The course of action and benchmark en

#### **5.3. Reordering, Summarization and then Answer query (on Specific Query)**

Summarization (Without Query Bias) enhances the RAG output when the summary is added in the context. Now, we want to see if reordering the context for both summary generation and RAG output enhances the generation further.

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The key points from the context are as follows:  1. The case revolves around a
tax dispute between Islamabad Electric Supply Company Limited (IESCO) and the
Appellate Tribunal Inland Revenue (ATIR) regarding tax demands for the years
2007-2012. 2. The petitioner argued that the lower fora failed to consider the
relevant provisions of law and passed orders based on assumptions, resulting in
double taxation. 3. The learned high court framed a question of law regarding
the invocation of machinery under Section 161 of the Income Tax Ordinance 2001
without recourse to Section 177. 4. The court held that Section 161 and Section
177 are independent provisions with self-governing corollaries, and the
commissioner may amend an order of recovery under Section 161 without prior
audit under Section 177. 5. The court also discussed Section 160, which deals
with the general provisions relating to advance payment of tax or deductions of
tax at source. 6. The court analyzed Section 177, which

#### **Experiments Conclude Here!**

* Summary + Redoerding with summary included in the context gives the best results and hence the fixed strategy for generation and would be used for ahead for comparison between Llama & Mistral ahead with ensemble retrieval as fixed best strategy for retrieval.

### **6. RAG with first LLM: Llama (Ensemble Retriever, Reordering Context, Summary Inclusion)**

#### **6.1. RAG with Llama: Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

generic_query = '''
What was court's judgement on the following income tax case:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022
'''

generic_query=generic_query.lower()

ensemble_context = ensemble_retriever.invoke(generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The key points from the context are:  1. The Islamabad Electric Supply Company
Limited (IESCO) filed income tax references against the Appellate Tribunal
Inland Revenue (ATIR) and the High Court of Islamabad. 2. The references
pertained to tax demands for the years 2007-2012, alleging non-deduction of
withholding tax and default surcharge. 3. The petitioner argued that the lower
forums failed to consider the requirement of initiating proceedings under
Section 177 of the Income Tax Ordinance 2001 before passing orders under
Sections 161 and 205. 4. The petitioner claimed that the orders were based on
assumptions and guesswork, and that they were imposed a tax liability without
definite information, resulting in double taxation. 5. The High Court framed a
common question of law, which was whether the respondent was justified in
holding proceedings and passing orders under Section 161/205 without proceedings
under Section 177 of the IT Ordinance. 6. The Income Tax Ordinance 2001 

#### **6.2. RAG with Llama: Specific Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The key points from the context are as follows:  1. The case revolves around a
tax dispute between Islamabad Electric Supply Company Limited (IESCO) and the
Appellate Tribunal Inland Revenue (ATIR) regarding tax demands for the years
2007-2012. 2. The petitioner argued that the lower fora failed to consider the
relevant provisions of law and passed orders based on assumptions, resulting in
double taxation. 3. The learned high court framed a question of law regarding
the invocation of machinery under Section 161 of the Income Tax Ordinance 2001
without recourse to Section 177. 4. The court held that Section 161 and Section
177 are independent provisions with self-governing corollaries, and the
commissioner may amend an order of recovery under Section 161 without prior
audit under Section 177. 5. The court also discussed Section 160, which deals
with the general provisions relating to advance payment of tax or deductions of
tax at source. 6. The court analyzed Section 177, which

#### **6.3. RAG with Llama: Completely Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

completely_generic_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205?

Which cases can you reference for this?

'''

completely_generic_query=completely_generic_query.lower()

ensemble_context = ensemble_retriever.invoke(completely_generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{completely_generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The context revolves around tax-related cases in Pakistan, specifically the
Income Tax Ordinance, 2001. The key points are:  1. A case (CP No. 2447-L/2022)
discusses the requirements for initiating proceedings under Section 111 of the
ordinance, which deals with unexplained income or assets. 2. The court rules
that a separate notice is required under Section 111, and a notice under Section
122(9) is not enough to initiate proceedings for amendment of assessment. 3.
Another case (CP No. 1417/2022) discusses the dismissal of an appeal on the
ground of default, and the court rules that the appellate tribunal can only
decide on merits and cannot dismiss the appeal in default. 4. A case (CP No.
1920/2022) discusses the initiation of proceedings under Section 161 and 205 of
the ordinance, which deals with default surcharge, and the court rules that
these proceedings can be initiated without complying with the prerequisites of
Section 177 of the ordinance. 5. The court also discusses

### **7. RAG with second LLM: Mistral (Ensemble Retriever, Reordering Context, Summary Inclusion)**

In [ ]:
# Note: The default behavior now has injection attack prevention off.

tokenizer2 = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", trust_remote_code=True)


model2 = AutoModelForCausalLM.from_pretrained(

    "mistralai/Mistral-7B-Instruct-v0.3",

    device_map="auto",

    trust_remote_code=True,

    torch_dtype=torch.bfloat16,

    quantization_config=BitsAndBytesConfig(load_in_8bit=True)

).eval()

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

time: 6min 54s (started: 2024-11-14 14:49:47 +00:00)


#### **7.1. RAG with Mistral: Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

generic_query = '''
What was court's judgement on the following income tax case:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022
'''

generic_query=generic_query.lower()

ensemble_context = ensemble_retriever.invoke(generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model2, tokenizer=tokenizer2)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary:  The case involves a dispute between Islamabad Electric Supply Company Limited
(IESCO) and the Appellate Tribunal Inland Revenue (Headquarter), Islamabad,
regarding income tax demands for the years 2007 to 2012. The petitioner argues
that the lower courts failed to consider that orders under sections 161 and 205
of the Income Tax Ordinance 2001 cannot be passed without initiating proceedings
under section 177 of the ordinance, which is a precondition before taking any
adverse action. The petitioner also contends that the impugned orders are based
on assumptions and guesswork, and that the petitioner was imposed a tax
liability without any definite information, leading to double taxation. The case
revolves around the question of law: whether under the facts and circumstances
of the case, the respondent no.1 was justified to hold proceedings and pass
orders under section 161/205 without proceedings under section 177 of the Income
Tax Ordinance 2001. The case is currently under a

#### **7.2. RAG with Mistral: Specific Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205? Use the following case for reference:

Case Title: M/s Islamabad Electric Supply Company Limited (IESCO) through its Finance Director Islamabad v. The Appellate Tribunal Inland Revenue (Headquarter), Islamabad through its Chairman Islamabad and others

Case No: C.P.L.A.1920/2022

'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model2, tokenizer=tokenizer2)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary:  The case involves a dispute between Islamabad Electric Supply Company Limited
(IESCO) and the Appellate Tribunal Inland Revenue (Headquarter), Islamabad,
regarding income tax demands for the years 2007 to 2012. The petitioner argues
that the lower courts failed to consider that orders under sections 161 and 205
of the Income Tax Ordinance 2001 cannot be passed without initiating proceedings
under section 177 of the ordinance, which is a precondition before taking any
adverse action. The High Court framed a common question of law to resolve the
difference of opinion: whether under the facts and circumstances of the case,
was the respondent justified to hold proceedings and pass orders under section
161/205 without proceedings under section 177 of the Income Tax Ordinance 2001?
The court's judgment concluded that the course of action and benchmark
enumerated under section 161 of the ordinance is not contingent upon the
compliance of pre-audit requirements mentioned under sectio

#### **7.3. RAG with Mistral: Completely Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

completely_generic_query='''

What are the implications of Section 177 of the Income Tax Ordinance regarding the audit exercise and its relation to Sections 161 and 205?

Which cases can you reference for this?

'''

completely_generic_query=completely_generic_query.lower()

ensemble_context = ensemble_retriever.invoke(completely_generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model2, tokenizer=tokenizer2)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{completely_generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary:  The context consists of several court judgments related to income tax laws in
Pakistan. The main points discussed in the judgments are:  1. In the case of
Commissioner Inland Revenue, Lahore v. M/s Millat Tractors Limited, the court
discusses the requirements for initiating proceedings for amendment of
assessments under section 122 of the ordinance based on information obtained
under section 111. The court holds that a separate notice under section 111 is
required before initiating proceedings under section 122.  2. In the case of
Farrukh Raza Sheikh v. The Appellate Tribunal Inland Revenue, the court
discusses the power of the Appellate Tribunal to dismiss an appeal in default of
appearance. The court notes that the power of the tribunal is confined to
deciding on the merits of the appeal and that dismissal for default of
appearance is not allowed.  3. In the case of Farrukh Raza Sheikh v. The
Appellate Tribunal Inland Revenue, the court also discusses section 132(2) of
the 

#### **LlaMa Wins**

LLaMA consistently outperforms Mistral in delivering comprehensive, relevant, and accurate responses. It excels in addressing queries thoroughly, maintaining focus on the context, and accurately interpreting legal and procedural details. For instance, in analyzing Section 177 of the Income Tax Ordinance and its relationship to Sections 161 and 205, LLaMA provided detailed explanations, cited case references (CP No. 1920/2022), and clarified legislative intent, showcasing its superior capability for complex queries. This makes LLaMA the preferred choice for reliable and detailed responses.

### **8. Final RAG Evaluation**

#### **8.1. Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

generic_query = '''
What was court's judgement on the following income tax case:

Case Title: Farrukh Raza Sheikh v. The Appellate Tribunal Inland Revenue, Lahore Bench through its Chairman and others

Case No: C.P.L.A.1417/2022
'''

generic_query=generic_query.lower()

ensemble_context = ensemble_retriever.invoke(generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: Here's a concise summary of the key points:  The case revolves around the powers
of the Appellate Tribunal Inland Revenue (Tribunal) in deciding tax appeals. The
key points are:  1. The jurisprudence in the subcontinent has consistently held
that the Tribunal must decide tax appeals on merits and not dismiss them in
default. 2. The Income Tax Act, 1922, and its corresponding sections, such as
Section 33(4), emphasize the Tribunal's obligation to decide appeals on merits.
3. The case of Chenniappa Mudaliar v. Commr. of Inc.-Tax (1964) 53 ITR 323, and
its subsequent upholding by the Supreme Court of India, established that the
Tribunal cannot dismiss an appeal in default, as it would put an end to the
appeal and prevent the High Court from performing its statutory duty of hearing
the tax reference. 4. The Appellate Tribunal Inland Revenue Rules, 2010, Rule
22(1), allows for the dismissal of tax appeals in default, which is challenged
as being ultra vires Section 132(2) of the In

#### **8.2. Specific Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

specific_query='''

What are the constitutional implications of not advertising honorary public positions like the COC slots, as discussed in the following case:

Case Title: Pakistan Electronic Media Regulatory Authority, Islamabad & another v. ARY Communications Limited, Karachi & others

Case No: C.P.L.A.1716/2022


'''

specific_query=specific_query.lower()

ensemble_context = ensemble_retriever.invoke(specific_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{specific_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The key points from the context are as follows:  1. The case revolves around the
appointment of the chairperson and members of the Council of Complaints (COC) of
the Pakistan Electronic Media Regulatory Authority (Pemra). 2. The High Court of
Sindh directed that the appointment of the chairperson and members of the COC
should be made after advertising the positions. 3. Pemra has filed a petition
for leave to appeal against this direction. 4. The issue at hand is whether
honorary appointments to public offices like the COC need to be made after
advertising the positions or if the government can headhunt and fill these
slots. 5. The Pemra Ordinance 2002 requires that the chairperson and members of
the COC must be citizens of eminence. 6. The government argues that it would be
difficult to find citizens of eminence if the positions are advertised, as they
do not usually apply for honorary positions. 7. The court is considering the
legislative will and the need to find citizens of

#### **8.3. Completely Generic Query**

In [ ]:
document_texts = [doc.page_content for doc in documents]

bm25_retriever = BM25Retriever.from_texts(
    document_texts, metadatas=[{}] * len(document_texts)
)

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_store.as_retriever()], weights=[0.5, 0.5])

completely_generic_query='''

Which cases discuss the interplay between procedural transparency, fairness in appointments, and adherence to statutory mandates in public or regulatory bodies?

'''

completely_generic_query=completely_generic_query.lower()

ensemble_context = ensemble_retriever.invoke(completely_generic_query, k=10)

# Reordering
reordering = LongContextReorder()
reordered_context_text = reordering.transform_documents(ensemble_context)

summary_prompt = f"""
context:
{reordered_context_text}

Provide a concise summary of the key points from the above context. Do not reference the query.
"""

# Summary
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False
}

messages_summary = [{"role": "user", "content": summary_prompt}]
summary_output = pipe(messages_summary, **generation_args)
summary_text = summary_output[0]['generated_text']

print("Summary:", textwrap.fill(summary_text, width=80))
print()

# Answer query
answer_prompt = f"""
Initial Summary:
{summary_text}

context:
{reordered_context_text}

Suppose you're a legal expert. Based on both the initial summary and the detailed context above, answer the following query:

{completely_generic_query}

Note: Only use the summary and context above to answer the query. Not all details may be necessary, so only refer to relevant points to answer the query.

If the context is insufficient to fully answer the query, respond with "I don't know as I don't have sufficient context!" and state why.
"""

messages_answer = [{"role": "user", "content": answer_prompt}]
output_answer = pipe(messages_answer, **generation_args)

print("Answer:", textwrap.fill(output_answer[0]['generated_text'], width=80))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Summary: The key points from the context are:  1. **Right to Fair Trial and Dignity**:
The right to a fair trial and dignity is a fundamental right, essential for
human dignity, and is guaranteed by Article 10A of the Constitution. 2.
**Independence and Impartiality**: The independence and impartiality of the
decision-maker are essential for a fair trial. 3. **Reasoned Judgment**: A
reasoned judgment is necessary for a fair trial, and without it, the right to a
fair trial would be compromised. 4. **Natural Justice**: The philosophy of
natural justice requires affording a right of audience before any detrimental
action is taken. 5. **Vested Rights**: Vested rights are created when
appointments are made with due process, and these rights cannot be withdrawn in
a perfunctory manner. 6. **Merit-Based Selection**: Merit should be the sole
criteria in the selection process for civil servants, and favoritism, nepotism,
and preferential treatment are inimical to good governance. 7. **Procedura

#### **RAG Strategy Stable and Successful**

Applying the final RAG through LlaMa on new judgements of different and similar themes, the outputs and scores turn out to be satisfactory based on the ground truth. The richness of the RAG output itself is also noteworthy. Therefore, the workflow and strategy employed in this project proves stable and successful in the implementation of a judicial RAG for SCP!

### **Future Work**

* **Metadata Embedding Strategy:** Instead of embedding metadata directly within document chunks, we can first combine the metadata into a single string and embed it separately. Then, for each document, we associate the chunk embeddings with their corresponding metadata embedding (i.e., group chunk embeddings under the same metadata embedding). This approach ensures that when a query is processed, it is first matched with the metadata embedding, and then the most relevant document chunks are retrieved based on this initial match. This strategy improves the matching precision by narrowing the query scope to relevant metadata before diving into document content.



* **Self-Querying and Auto-Retrieval for Metadata:** To enhance the search process, a self-querying or auto-retrieval mechanism can be introduced. This mechanism would analyze the user's query and automatically generate and apply relevant metadata tags and filters. If no metadata tags or filters are provided by the user, the system can skip the metadata matching step and directly proceed to matching the query with the document chunk embeddings. This approach enhances flexibility and ensures that the search system can still operate effectively even without explicit metadata input.



* **Embedding Model with Large Dimension:** A potential direction for future work is to use an embedding model with higher dimensions, such as 2048 or more, to better capture the complexities of legal texts. Legal documents often involve specialized terminology and context-dependent details, and a richer representation could improve retrieval accuracy. This approach would enhance the system’s ability to handle ambiguous terms and case-specific jargon, leading to more precise and relevant results in legal queries. This will also help address the issue of repeated phrases or terms within legal documents. With a richer representation, the model can better capture the nuances and relationships between repeated phrases, allowing it to differentiate between contexts where the same term is used in different ways.





